# IndoXTC - Fine-tune Head Toxic [XLM-R] [ALL]
Exploring Indonesian hate speech/abusive & sentiment text classification using multilingual language model.

This kernel is a part of my undergraduate final year project.
Checkout the full github repository:
https://github.com/ilhamfp/indonesian-text-classification-multilingual

In [1]:
#############################
# Experiment configurations #
#############################

INDO_DATA_NAME = 'toxic'
DATA_PATH_INDO = '../input/indoxtc-extracting-toxic-features-xlm-r-simple'

FOREIGN_DATA_NAME = 'toxic'
DATA_PATH_FOREIGN = '../input/indoxtc-combining-toxic-en-features-xlm-r'

MODEL_NAME = 'XLM_R'

EXPERIMENT_TYPE_LIST = ['A', 'B', 'C'] # A / B / C
TOTAL_DATA_LIST = [500, 1000, 2500, 5000, 7500, 11852]
FOREIGN_LANG_DATA_MULT_LIST = [0.25, 0.5, 0.75, 1, 1.5, 2, 3, 4, 5, 6, 7, 8, 9, 10]
RANDOM_SEED_LIST = [4,5,6]
VALIDATION_DATA = 0.1

In [2]:
import os
import numpy as np
import pandas as pd
from load_data import load_experiment_features, set_random_seed_data
from model_head import set_seed, train, test, evaluate

CUDA is not available.  Training on CPU ...


## Experiment

In [3]:
result_list_final = []
for RANDOM_SEED in RANDOM_SEED_LIST:
    print("\n############################")
    print("### START RANDOM_SEED {} ###".format(RANDOM_SEED))
    print("############################")
    result_list = []
    for TOTAL_DATA in TOTAL_DATA_LIST:
        print("\n##### TOTAL_DATA {} #####".format(TOTAL_DATA))

        for EXPERIMENT_TYPE in EXPERIMENT_TYPE_LIST:
            for FOREIGN_LANG_DATA_MULT in FOREIGN_LANG_DATA_MULT_LIST:
                set_seed(seed=RANDOM_SEED)
                set_random_seed_data(seed=RANDOM_SEED)
                if EXPERIMENT_TYPE != 'C':
                    print('\n~~Result_{}_{}~~'.format(EXPERIMENT_TYPE,
                                                       TOTAL_DATA))
                else:
                    print('\n~~Result_{}_{}_{}~~'.format(EXPERIMENT_TYPE,
                                                   TOTAL_DATA,
                                                   FOREIGN_LANG_DATA_MULT))

                train_loader, valid_loader, test_loader = load_experiment_features(DATA_PATH_INDO,
                                                                                   DATA_PATH_FOREIGN,
                                                                                   tipe=EXPERIMENT_TYPE, 
                                                                                   total_data=TOTAL_DATA, 
                                                                                   foreign_mult=FOREIGN_LANG_DATA_MULT,
                                                                                   valid_size=VALIDATION_DATA)

                train(train_loader, valid_loader, learning_rate=0.0001)
                print("## Test phase...")
                y_true, y_pred_proba = test(test_loader)
                max_f1, max_recall, max_precision, max_threshold = evaluate(y_true, 
                                                                            y_pred_proba,
                                                                            threshold=0.5)

                result = pd.DataFrame({
                    'y_true': y_true,
                    'y_pred_proba': y_pred_proba
                })
                
                OUTPUT_FOREIGN_LANG_DATA_MULT = FOREIGN_LANG_DATA_MULT
                if EXPERIMENT_TYPE == 'A':
                    OUTPUT_FOREIGN_LANG_DATA_MULT = 0
                elif EXPERIMENT_TYPE == 'B':
                    OUTPUT_FOREIGN_LANG_DATA_MULT = -1
                
                OUTPUT_TOTAL_FOREIGN_DATA = int(TOTAL_DATA*FOREIGN_LANG_DATA_MULT)
                if EXPERIMENT_TYPE == 'A':
                    OUTPUT_TOTAL_FOREIGN_DATA = 0
                elif EXPERIMENT_TYPE == 'B':
                    OUTPUT_TOTAL_FOREIGN_DATA = TOTAL_DATA

                result_list.append({
                    'type':EXPERIMENT_TYPE,
                    'total_data':TOTAL_DATA, 
                    'seed':RANDOM_SEED,
                    'foreign_mult':OUTPUT_FOREIGN_LANG_DATA_MULT,
                    'total_foreign_data':OUTPUT_TOTAL_FOREIGN_DATA,
                    'f1':max_f1,
                    'recall':max_recall,
                    'precision':max_precision
                })

                os.remove("model.pt")

                if EXPERIMENT_TYPE != 'C':
                    break
                    
    final_result = pd.DataFrame(result_list) 
    final_result = final_result.sort_values(by=['total_data', 'foreign_mult'])
    final_result.to_csv('final_{}_{}_{}_result_combined_seed_{}.csv'.format(INDO_DATA_NAME,
                                                                           FOREIGN_DATA_NAME,
                                                                           MODEL_NAME,
                                                                           RANDOM_SEED),
                        index=False)
    
    result_list_final.extend(result_list)


############################
### START RANDOM_SEED 4 ###
############################

##### TOTAL_DATA 500 #####

~~Result_A_500~~
EarlyStopping! Epoch 37
Last: 24 	T-Loss: 0.680009 	T-Acc: 0.564444 	V-Loss: 0.667340 	V-Acc: 0.540000
## Test phase...
Final test Loss: 0.679882
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.35661944 	Recall: 0.99863201 	Prec: 0.55471125

~~Result_B_500~~
EarlyStopping! Epoch 37
Last: 24 	T-Loss: 0.692064 	T-Acc: 0.482222 	V-Loss: 0.672984 	V-Acc: 0.640000
## Test phase...
Final test Loss: 0.694904
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.31780159 	Recall: 0.00957592 	Prec: 0.77777778

~~Result_C_500_0.25~~
EarlyStopping! Epoch 52
Last: 39 	T-Loss: 0.672653 	T-Acc: 0.586146 	V-Loss: 0.667274 	V-Acc: 0.620000
## Test phase...
Final test Loss: 0.674983
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0

/opt/conda/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


EarlyStopping! Epoch 110
Last: 97 	T-Loss: 0.516818 	T-Acc: 0.761860 	V-Loss: 0.522458 	V-Acc: 0.764557
## Test phase...
Final test Loss: 0.509545
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.76392949 	Recall: 0.83720930 	Prec: 0.76884422

~~Result_B_11852~~


/opt/conda/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


EarlyStopping! Epoch 25
Last: 12 	T-Loss: 0.647762 	T-Acc: 0.717165 	V-Loss: 0.681849 	V-Acc: 0.585654
## Test phase...
Final test Loss: 0.678890
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.57308576 	Recall: 0.47469220 	Prec: 0.66221374

~~Result_C_11852_0.25~~


...

EarlyStopping! Epoch 58
Last: 45 	T-Loss: 0.493639 	T-Acc: 0.766888 	V-Loss: 0.573347 	V-Acc: 0.722363
## Test phase...
Final test Loss: 0.555667
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.70212000 	Recall: 0.75102599 	Prec: 0.72908367

~~Result_C_11852_9~~


/opt/conda/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


EarlyStopping! Epoch 51
Last: 38 	T-Loss: 0.490977 	T-Acc: 0.769519 	V-Loss: 0.579958 	V-Acc: 0.698734
## Test phase...
Final test Loss: 0.559991
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.70043882 	Recall: 0.73050616 	Prec: 0.73452545

~~Result_C_11852_10~~


/opt/conda/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


EarlyStopping! Epoch 56
Last: 43 	T-Loss: 0.492485 	T-Acc: 0.768464 	V-Loss: 0.584503 	V-Acc: 0.690295
## Test phase...
Final test Loss: 0.566218
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.69685013 	Recall: 0.73461012 	Prec: 0.72862958


## Save

In [4]:
final_result = pd.DataFrame(result_list_final) 
final_result = final_result.sort_values(by=['total_data', 'foreign_mult'])
final_result.to_csv('final_{}_{}_{}_result_combined.csv'.format(INDO_DATA_NAME,
                                                               FOREIGN_DATA_NAME,
                                                               MODEL_NAME),
                        index=False)


In [5]:
print(final_result.shape)
final_result.head()

(288, 8)


,type,total_data,seed,foreign_mult,total_foreign_data,f1,recall,precision
1,B,500,4,-1.0,500,0.317802,0.009576,0.777778
97,B,500,5,-1.0,500,0.585923,0.629275,0.631868
193,B,500,6,-1.0,500,0.413612,0.120383,0.727273
0,A,500,4,0.0,0,0.356619,0.998632,0.554711
96,A,500,5,0.0,0,0.384169,0.998632,0.561107
